In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd 

import h2o
from h2o.automl import H2OAutoML

TRAIN_PATH = "../input/tabular-playground-series-may-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-may-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-may-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv "

ID = "id"
TARGET = "target"

NEW_TRAIN_PATH = "train.csv"
NEW_TEST_PATH = "test.csv"

SEED_LIST = [7,77]
MAX_RUNTIME_SECS = 60 * 3


In [3]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

for df in [train, test]:
    # Extract the 10 letters from f_27 into individual features
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
train.to_csv(NEW_TRAIN_PATH,index=False)
test.to_csv(NEW_TEST_PATH,index=False)


In [4]:
h2o.init()

train = h2o.import_file(NEW_TRAIN_PATH)
test = h2o.import_file(NEW_TEST_PATH)

x = train.columns
y = TARGET

x.remove(y)
x.remove(ID) #remove id  

pred_test = []
for selSeed in SEED_LIST:
    aml_y = H2OAutoML(max_runtime_secs=MAX_RUNTIME_SECS, seed=selSeed)
    aml_y.train(x=x, y=y, training_frame=train)

    preds_y = aml_y.predict(test)
    pred_test.append(preds_y.as_data_frame().predict) 


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8ftspvr6
  JVM stdout: /tmp/tmp8ftspvr6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8ftspvr6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 26 days
H2O_cluster_name:,H2O_from_python_unknownUser_t0ue3k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.97 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
08:12:25.924: _train param, Dropping bad and constant columns: [f_27]
08:12:25.924: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

███████████████████
08:13:19.154: _train param, Dropping bad and constant columns: [f_27]
08:13:19.154: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


08:13:21.66: _train param, Dropping bad and constant columns: [f_27]
08:13:21.66: _response param, We have detected that your response column has only 2 unique valu

/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


AutoML progress: |
08:15:28.860: _train param, Dropping bad and constant columns: [f_27]
08:15:28.860: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

███████████████████
08:16:22.28: _train param, Dropping bad and constant columns: [f_27]
08:16:22.28: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


08:16:23.463: _train param, Dropping bad and constant columns: [f_27]
08:16:23.463: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

█████████████████
08:17:13.165: _train param, Droppin

/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [5]:
from scipy import stats
modeResult = stats.mode(pred_test, axis=0)
final_test_pred = modeResult.mode

submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] = final_test_pred[0]
submission.to_csv(SUBMISSION_PATH, index=False)
submission.head()


,id,target
0,800000,0.772326
1,800001,0.772326
2,800002,0.772326
3,800003,0.772326
4,800004,0.772326
